# [IAPR][iapr]: Project


**Group ID:** 32

**Author 1 (sciper):** Ghali CHRAIBI (262251)  
**Author 2 (sciper):** Yann Yasser HADDAD (272292)   
**Author 3 (sciper):** Julien BERGER (xxxxx)   

**Release date:** 07.05.2021  
**Due date:** 03.06.2021 (23h59)


## Important notes

The lab assignments are designed to teach practical implementation of the topics presented during class as well as preparation for the final project, which is a practical project which ties together the topics of the course. 

As such, in the lab assignments/final project, unless otherwise specified, you may, if you choose, use external functions from image processing/ML libraries like opencv and sklearn as long as there is sufficient explanation in the lab report. For example, you do not need to implement your own edge detector, etc.

**! Before handling back the notebook !** rerun the notebook from scratch `Kernel` > `Restart & Run All`


[iapr]: https://github.com/LTS5/iapr

## Imports 

In [ ]:
pip install plotly

In [1]:
import warnings
warnings.filterwarnings('ignore')
import plotly.express as px
import plotly.graph_objects as go
import skimage.io
import pickle
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from utils import print_results, evaluate_game
from skimage.color import label2rgb, gray2rgb, rgb2gray, rgb2hsv
from skimage.transform import resize, rotate, rescale
from skimage.util import img_as_ubyte, crop
from skimage.filters import (laplace, prewitt, sobel, roberts, median,
gaussian, threshold_otsu, threshold_multiotsu,  difference_of_gaussians,
threshold_isodata, threshold_mean, threshold_yen, threshold_sauvola,
threshold_niblack, threshold_triangle, threshold_li, threshold_local)
from skimage.segmentation import clear_border
from skimage.measure import label, regionprops, regionprops_table, find_contours
from skimage.morphology import (disk, square, closing, binary_opening,
binary_closing, opening, binary_dilation, binary_erosion, remove_small_objects)

ModuleNotFoundError: No module named 'plotly'

In [4]:
pip install torch

^C
Note: you may need to restart the kernel to use updated packages.


In [3]:
# Torch
import torch
from torch import nn
import torch.nn.functional as F

ModuleNotFoundError: No module named 'torch'

---
## 0. Introduction

An anonymous researcher that we will name Lann Yecun is convinced that the MNIST dataset still has great potential. He decides to create a playing card game based on MNIST digits and different figures. The game uses a standard 52 card deck which is composed of four French suits/colours: clubs (&#9827;), diamonds (&#9830;), hearts (&#9829;) and spades (&#9824;). Each suit includes 10 digit cards (from 0 to 9) and 3 figures (Jack-J, Queen-Q, and King-K). Here is an example of the 13 spade cards with their name.


<img src="data/media/example_cards.png">


We can find the same arrangement of cards for the clubs, diamonds, and hearts. 


## 1. Rules


### 1.1 Standard

The rules are based on the simple battle card game. The goal of the game is to win as many points as possible. Each turn, the 4 players play a card in front of them. As displayed in the example below. The rules are the following:

- The cards are ranked in the following order : **0 < 1 < 2 < 3 < 4 < 5 < 6 < 7 < 8 < 9 < J < Q < K**.
- The player with the highest-ranked card wins the round and obtains **1 point**. 
- If the highest-ranked card is the same for multiple players we call it a draw and all winners get **1 points**. 
- In this configuration, we **do not** take into account the suits. The game only rely on the card ranks. 
- The game lasts 13 rounds. After the last round, the winner is the player that has the largest number of points. 
- In the example below Player 1 wins the round with his Queen ( 0 < 8 < J < **Q**).

If two or more players have the same number of points they share the victory.

### 1.2 Advanced

The advanced rules take into account the suits. 

- At the beginning of **each round** a random player is designated as the **dealer**. The dealer places a green token with the letter *D* next to him (player 1 in the example below).
- Only the cards that belong to the same suit as the one of the dealer are considered valid. In the example below, only Player 4 is competing with Player 1 as spade was selected by the dealer (e.i., Player 1). Player 2 and 3 are out for this round. Player 1 wins the round and **1 point** with the Queen ( 0&#9824; < **Q&#9824;**).
- There cannot be any draw between the players as they are not any card duplicates.
- We use the same system as the standard method to count the points.


<img src="data/media/example_round.jpg">


### 1.3 Notes

- The orientation of the card is linked to the position of the player around the table. For instance, to read the card of the 3rd player you will have to rotate it by 180°.
- The **digits** always **face** the players around the table. The figures can have random orientations.
- Player 1 **always** seats south of the table. The players are **always** ordered counter-clockwise as in the example. 
- The dealers can change between the rounds and games.
- Some cards **might** apear multiple times per game.
- Pictures are always taken from rougthly the same altitude.
- The digits from the training set **would not** be the same as the one of the testing set.

---
## 2. Data

You will be given the images of 7 games that were played ([download link](https://drive.google.com/drive/folders/1fEy27wnJsUJPRsEEomzoAtP56s-7HFtk?usp=sharing)). The data are composed of:
   - 7 folder named after the games (game1 to game7).
   - Each game includes 13 ordered images (1st to 13th round).
   - Each game includes a csv file with the ground truth of the game. The first row list the players (P1 to P4) as well as the dealer (D). The following rows represent the rounds (1 to 13). We represent the card played with 2 character as $AB$ where $A \in [0-9, J, Q, K]$ is the rank of the card and $B \in [C, D, H, S]$ is the suit. For example, QS means "(Q)ueen of (S)pade" and 0D means "(0) of (D)iamond". The dealer is represented by the ID of the player (e.g. P1 -> 1).
   
You are free to use external datasets such as the original MNIST train set that you used in lab 3.

### Data loading

In [ ]:
train_data = 'data/train_games'
train_game_count = 7
game_round_count = 13

dict_data = {}

for i in range(1, train_game_count+1):
    # For each game, store the path of the csv and the image of each round
    dict_data[f'game{i}'] = {}
    dict_data[f'game{i}']['url'] = train_data + f'/game{i}'
    dict_data[f'game{i}']['csv'] = train_data + f'/game{i}.csv'
    
    for j in range(1, game_round_count+1):
        dict_data[f'game{i}'][f'round{j}'] = skimage.io.imread(train_data + f'/game{i}/{j}.jpg')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8,8))
ax.imshow(dict_data['game3']['round7'])
ax.set_title('Game1 Round1')
ax.axis('off')
plt.show()

---
## 3. Your Tasks

Your task is to ready yourself for the final evaluation. The day of the exam we will give you a new folder with a new game. ! The digits on the cards **differ** from the one of the traning set. When given a new data folder with 13 images your should be able to:

**Task 0**
   - Plot an overlay for each round image that shows your detections and classification. You can for example plot bounding boxes around the cards/dealer token and add a text overlay with the name of the classes.

**Task 1**
   - (a) Predict the **rank** of the card played by each player at each round (Standard rules).
   - (b) Predict the **number of points** of each player according to **Standard** rules
 
**Task 2**
   - (a) Detect which player is the selected **dealer** for each round.
   - (b) Predict the **rank** and the **suit** of the card played by each player at each round (Advanced rules).
   - (c) Predict the **number of points** of each player according to **Advanced** rules

---

**Before the exam (until 03.06.21 at 23h59)**
   - Create a zipped folder named **group_xx.zip** that you upload on moodle (xx being your group number).
   - Include a **runnable** code (Jupyter Notebook and external files) and your presentation in the zip folder.
   
**The day of the exam (04.06.21)**
   - You will be given a **new folder** with 13 images (rounds) and but **no ground truth** (csv file).
   - We will ask you to run your pipeline in **realtime** and to send us your prediction of task 1 and 2 that you obtain with the function **print_results**. 
   - On our side we will compute the perfomance of your classification algorithm. 
   - To evaluate your method we will use the **evaluate_game** function presented below. To understand how the provided functions work please read the documentation of the functions in **utils.py**.
   - **Please make sure your function returns the proper data format to avoid points penalty the day of the exam**. 

### Segmentation Functions

In [ ]:
#Input: coloured image of the round
#Output: DataFrame containing all segmented objects (cards + dealer + random objects)
#with geometrical properties

def find_potential_objects_in_original_round_image(coloured_image):
    hsv_img = rgb2hsv(coloured_image) 
    hsv_img = img_as_ubyte(hsv_img) # Convert 0-1 to 0-255 
    
    thres_brightness = threshold_multiotsu(hsv_img[:,:,2]).max() # threshold for the "value" channel
    
    # HSV image is filtered on the 3 channels:
    # H: [145-200] white colour , S: [0-255] , V: [threshold-255] high values
    filtered_im = cv2.inRange(hsv_img, np.array([145,0,thres_brightness]), np.array([200,255,255])) # Binary filtered image
    
    labeled_im = label(filtered_im, background=None, connectivity=filtered_im.ndim) # Find objects using region growing and labeling
    labeled_im = remove_small_objects(labeled_im, min_size=30000) # Remove objects smaller than the dealer and the cards
    labeled_im = median(labeled_im, square(15)) # Remove noise
    
    # Dictionnary that stores information for each segmented objects 
    props_objs = regionprops_table(labeled_im, properties=(
                                                  "label",
                                                  "area",
                                                  "filled_area",
                                                  "major_axis_length",
                                                  "minor_axis_length",
                                                  'centroid',
                                                  "slice",
                                                  "image"
                                                   ))
    df_objs = pd.DataFrame(props_objs)
    df_objs["labeled_im"] = 0 # random value
    df_objs["labeled_im"] = df_objs["labeled_im"].apply(lambda x: labeled_im)
    
    
    return df_objs
    
    

In [ ]:
#Input: DataFrame containing all the potential objects 
#Output: DataFrame with 4 objects corresponding to the 4 cards (unordered)

def select_cards_from_potential_objects(df_objs): 
    
    # Features differentiating cards from the rest of the objects
    # These are reference values obtained by exploring the dataset
    C_REF_MAJOR_AXIS_MEAN = 838
    C_REF_MAJOR_AXIS_STD = 16
    C_REF_MINOR_AXIS_MEAN = 542
    C_REF_MINOR_AXIS_STD = 23
    
    cards_feature1_series = (df_objs["major_axis_length"]-C_REF_MAJOR_AXIS_MEAN)/C_REF_MAJOR_AXIS_STD # Normalized feature 1 
    cards_feature2_series = (df_objs["minor_axis_length"]-C_REF_MINOR_AXIS_MEAN)/C_REF_MINOR_AXIS_STD # Normalized feature 2
    
    # Euclidian distance corresponding to the similarity between an object and the reference card.
    # The smaller the distance, the greater the similarity. Here the 4 most similar objects are kept.
    cards_similarity_distances_series = (cards_feature1_series**2+cards_feature2_series**2)**0.5 # Euclidian distance
    df_objs["card_similarity_measure"] = cards_similarity_distances_series
    
    return df_objs.nsmallest(4, "card_similarity_measure")


In [ ]:
#Input: DataFrame containing all the potential objects 
#Output: Series with 1 object corresponding to the dealer

def select_dealer_from_potential_objects(df_objs): 
    
    # Features differentiating the dealer from the rest of the objects
    # These are reference values obtained by exploring the dataset
    D_REF_AREA_MEAN = 45061
    D_REF_AREA_STD = 1202
    D_REF_FILLED_AREA_MEAN = 65325
    D_REF_FILLED_AREA_STD = 1134

    dealer_feature1_series = (df_objs["area"]-D_REF_AREA_MEAN)/D_REF_AREA_STD # Normalized feature 1 
    dealer_feature2_series = (df_objs["filled_area"]-D_REF_FILLED_AREA_MEAN)/D_REF_FILLED_AREA_STD # Normalized feature 2
    
    # Euclidian distance corresponding to the similarity between an object and the reference dealer.
    # The smaller the distance, the greater the similarity. Here the most similar object is kept.
    dealer_similarity_distances_series = (dealer_feature1_series**2+dealer_feature2_series**2)**0.5 # Euclidian distance
    df_objs["dealer_similarity_measure"] = dealer_similarity_distances_series
    
    return df_objs.nsmallest(1, "dealer_similarity_measure").iloc[0] #Return a series

In [ ]:
#Input: Original image of the round; DataFrame containing the 4 card objects
#Output: Ordered DataFrame (index 0 = player 1; index 1 = player 2; index 2 = player 3; index 3 = player 4)
#that contains the extracted cards (colour and binary version). Extracted cards are also rotated depending on
#the player position

def extract_cards_from_original_round_image(coloured_image, cards_only_df):

    ordered_cards_only_df = pd.DataFrame()
    
    # Players are found using the centroid of the card object. Centroid = (x0, y0)
    player1_series = cards_only_df.loc[cards_only_df["centroid-0"].idxmax()] # Player 1 has the highest x0
    
    # The "slice" property is used to extract the card from the original image
    player1_series["image_coloured"] = coloured_image[player1_series["slice"]] 
   
    player2_series = cards_only_df.loc[cards_only_df["centroid-1"].idxmax()] # Player 2 has the highest y0
    
    # The "image" property is used to extract the card from the original image in binary version
    player2_series["image"] = rotate(player2_series["image"], -90, resize=True) # Rotation of the extracted binary card
    player2_series["image_coloured"] = rotate(coloured_image[player2_series["slice"]], -90, resize=True)

    player3_series = cards_only_df.loc[cards_only_df["centroid-0"].idxmin()] # Player 3 has the lowest x0
    player3_series["image"] = rotate(player3_series["image"], 180, resize=True)
    player3_series["image_coloured"] = rotate(coloured_image[player3_series["slice"]], 180, resize=True)

    player4_series = cards_only_df.loc[cards_only_df["centroid-1"].idxmin()] # Player 4 has the lowest y0
    player4_series["image"] = rotate(player4_series["image"], 90, resize=True)
    player4_series["image_coloured"] = rotate(coloured_image[player4_series["slice"]], 90, resize=True)


    ordered_cards_only_df = (ordered_cards_only_df.append(player1_series, ignore_index=True)
                             .append(player2_series, ignore_index=True)
                             .append(player3_series, ignore_index=True)
                             .append(player4_series, ignore_index=True))
    
    ordered_cards_only_df["player"] = [1,2,3,4]
    
    return ordered_cards_only_df
    

In [ ]:
#Input: DataFrame containg the extracted cards in order (index 0 = player 1; index 1 = player 2; index 2 = player 3; index 3 = player 4)
# and the dealer series 
#Output: Same DataFrame with a new column called "dealer" (with True and False values)
def Add_dealer_status_to_extracted_cards(ordered_cards_only_df, dealer_series):
    
    dealer_centr0 = dealer_series["centroid-0"]
    dealer_centr1 = dealer_series["centroid-1"]
   
    # To find the dealer, the euclidian distance between the centroids of the cards and the dealer is computed 
    ordered_cards_only_df["dealer"] = [False,False,False,False]
    distances_to_dealer_series = ((ordered_cards_only_df["centroid-0"]-dealer_centr0)**2 + (ordered_cards_only_df["centroid-1"]-dealer_centr1)**2)**0.5 # Euclidian distance
    ordered_cards_only_df.loc[distances_to_dealer_series.idxmin(),"dealer"] = True
    
    return ordered_cards_only_df

### Plotting functions

In [ ]:
#Input: coloured image of the round
#Output: Series containing a summary of the results (an extracted card for each player + attribution of the dealer) 
def plot_overlay_for_round_image(coloured_image):
    
    df_objs = find_potential_objects_in_original_round_image(coloured_image)
    
    cards_only_df = select_cards_from_potential_objects(df_objs)
    dealer_series = select_dealer_from_potential_objects(df_objs)
    
    ordered_cards_only_df = extract_cards_from_original_round_image(coloured_image, cards_only_df)
    ordered_cards_only_df = Add_dealer_status_to_extracted_cards(ordered_cards_only_df, dealer_series)
    
    
    fig = px.imshow(rgb2gray(coloured_image), binary_string=True)
    fig.update_traces(hoverinfo='skip') # hover is only for label info
    properties = ['player', 'dealer']

    # For each label, add a filled scatter trace for its contour,
    # and display the properties of the label in the hover of this trace.
    result_series = pd.Series() # store information of the round. This is the output of the function
    
    for index in range(4):
        result_series[f"P{index+1}_extracted_card"] = (ordered_cards_only_df.iloc[index]["image_coloured"], ordered_cards_only_df.iloc[index]["image"])
        
        label_ = ordered_cards_only_df.iloc[index]["label"]
        contour = find_contours(ordered_cards_only_df.iloc[index]["labeled_im"] == label_, 0.5)[0]
        y, x = contour.T
        hoverinfo = ''
        for prop_name in properties:
            hoverinfo += f'<b>{prop_name}: {ordered_cards_only_df.iloc[index][prop_name]}</b><br>'
        fig.add_trace(go.Scatter(
            x=x, y=y, #name = label_
            mode='lines', fill='toself', showlegend=False,
            hovertemplate=hoverinfo, hoveron='points+fills'))
    
    result_series["dealer"] = int(ordered_cards_only_df[ordered_cards_only_df["dealer"]]["player"][0]) #
    
    label_dealer = dealer_series["label"]
    contour_dealer = find_contours(dealer_series["labeled_im"] == label_dealer, 0.5)[0]
    y_d, x_d = contour_dealer.T
    fig.add_trace(go.Scatter(
        x=x_d, y=y_d, name = "dealer",
        mode='lines', fill='toself', showlegend=False,
        hovertemplate=" ", hoveron='points+fills'))

    fig.show()
    return result_series

In [ ]:
#Input: dict_data, game_number
#Output: DataFrame containing a summary of the results for each round (an extracted card for each player + attribution of the dealer) 
def plot_overlay_for_all_round_images(dict_data, game_number=1):
    
    results_game = pd.DataFrame()
    for i in range(13):
        coloured_image = dict_data[f"game{game_number}"][f"round{i+1}"]
        print(f"The segmentation of cards and dealer for round {i+1} is:")
        results_round = plot_overlay_for_round_image(coloured_image)
        results_game = results_game.append(results_round, ignore_index=True)
    
    return results_game
    

In [ ]:
#Input: coloured image of the round
#Output: no output. Just the image in HSV 
def plot_round_image_in_HSV(coloured_image):
        print("HSV channels of the image:")
        hsv_img = rgb2hsv(coloured_image) 
        hsv_img = img_as_ubyte(hsv_img) # Convert 0-1 to 0-255 
        fig = px.imshow(hsv_img, binary_string=True)
        fig.show()
    

### Functions to exctract numbers or suits

In [ ]:
#Input: extracted Coloured card and the exact same card in binary format (obtained using the ["image"] property)
#Output: cropped gray-scaled image of the numberin the mnist format
def create_mnist_number_from_extracted_coloured_cards(coloured_card, binary_card):
    
    cropped_binary_card = crop(binary_card, ((binary_card.shape[0]*0.15, binary_card.shape[0]*0.15), (binary_card.shape[1]*0.15, binary_card.shape[1]*0.15)))
    cropped_coloured_card = crop(coloured_card, ((coloured_card.shape[0]*0.15, coloured_card.shape[0]*0.15), (coloured_card.shape[1]*0.15, coloured_card.shape[1]*0.15), (0, 0)))

    # Take the negative of the segmented binary card (numbers become objects instead of background)
    filt1 = (cropped_binary_card == False)
    filt2 = (cropped_binary_card == True)
    cropped_binary_card[filt1] = True
    cropped_binary_card[filt2] = False   
    cropped_binary_card = binary_dilation(cropped_binary_card, square(45)) # Closes the number having holes i.e some 5 and 2
    labeled_im = remove_small_objects(label(cropped_binary_card), 5000)
    
    props_objs = regionprops_table(labeled_im, properties=(
                                                  "label",
                                                  "area",
                                                  'centroid',"slice"))
    df_objs = pd.DataFrame(props_objs)
    coloured_cropped_number = cropped_coloured_card[df_objs.loc[df_objs["area"].idxmax()]["slice"]] # extract the number (biggest object)
    
    
    cropped_number_gray = rgb2gray(coloured_cropped_number)
    thresh = threshold_otsu(cropped_number_gray)
    return resize(img_as_ubyte(cropped_number_gray < thresh), (28,28))
 
#This function can be deleted...
#Input: extracted Coloured card in binary format (obtained using the ["image"] property)
#Output: cropped gray-scaled image of the number in the mnist format
def create_mnist_number_from_extracted_binary_cards(binary_card):
    
    cropped_binary_card = crop(binary_card, ((binary_card.shape[0]*0.15, binary_card.shape[0]*0.15), (binary_card.shape[1]*0.15, binary_card.shape[1]*0.15)))

    # Take the negative of the segmented binary card (numbers become objects instead of background)
    #filt1 = (cropped_binary_card == False)
    #filt2 = (cropped_binary_card == True)
    #cropped_binary_card[filt1] = True
    #cropped_binary_card[filt2] = False   
    cropped_binary_card_ = binary_dilation(cropped_binary_card, square(45)) # Closes the number having holes i.e some 5 and 2
    labeled_im = remove_small_objects(label(cropped_binary_card_), 5000)
    
    props_objs = regionprops_table(labeled_im, properties=(
                                                  "label",
                                                  "area",
                                                  'centroid',"slice"))
    df_objs = pd.DataFrame(props_objs)
    binary_cropped_number = cropped_binary_card[df_objs.loc[df_objs["area"].idxmax()]["slice"]] #extract the number (biggest object)
    
    return resize(binary_cropped_number, (28,28))
    


In [ ]:
#Input: extracted Coloured card 
#Output: cropped gray-scaled image of the suit in the mnist format
def create_mnist_suit_from_extracted_coloured_cards(coloured_card, upper_left=True):
    if (upper_left):
        cropped_coloured_card = crop(coloured_card, ((0, coloured_card.shape[0]*0.6), (0, coloured_card.shape[1]*0.5), (0, 0)))
    else:
        cropped_coloured_card = crop(coloured_card, ((coloured_card.shape[0]*0.6, 0), (coloured_card.shape[1]*0.5, 0), (0, 0)))

    cropped_coloured_card_gray = rgb2gray(cropped_coloured_card)
    thresh = threshold_otsu(cropped_coloured_card_gray)
    cropped_coloured_card_gray_binary = img_as_ubyte(cropped_coloured_card_gray < thresh)
    labeled_im = label(cropped_coloured_card_gray_binary)
    labeled_im = clear_border(labeled_im)
    #labeled_im = remove_small_objects(labeled_im, 1000)
    #labeled_im = median(labeled_im, rect(5))

    
    props_objs = regionprops_table(labeled_im, properties=(
                                                  "label",
                                                  "area",
                                                  'centroid',"slice"))
    df_objs = pd.DataFrame(props_objs)
    extracted_suit_coloured = cropped_coloured_card[df_objs.loc[df_objs["area"].idxmax()]["slice"]]
    
    extracted_suit_gray = rgb2gray(extracted_suit_coloured)
    thresh = threshold_otsu(extracted_suit_gray)
    return resize(img_as_ubyte(extracted_suit_gray < thresh), (28,28))

### Functions used to create training sets for number and suit recognition
#### Cells are in MARKDOWN because there is no need to run them again. It takes time (30min) and will overwrite already existing files

#Input: dict_data
#Output: Series with each element being a tuple ("3H", mnist_image_method1, mnist_image_method2)
def create_training_set_for_numbers(dict_data):
    
    final_series = pd.Series()
    for game_number in range(7):
        player1_cards_coloured = []
        player2_cards_coloured = []
        player3_cards_coloured = []
        player4_cards_coloured = []

        player1_cards_binary = []
        player2_cards_binary = []
        player3_cards_binary = []
        player4_cards_binary = []

        for i in range(13):
            coloured_image = dict_data[f'game{game_number+1}'][f'round{i+1}']

            df_objs = find_potential_objects_in_original_round_image(coloured_image)
            cards_only_df = select_cards_from_potential_objects(df_objs)      
            cards_ordered_df = extract_cards_from_original_round_image(coloured_image, cards_only_df)


            player1_cards_coloured.append(cards_ordered_df.loc[0]["image_coloured"])
            player2_cards_coloured.append(cards_ordered_df.loc[1]["image_coloured"])
            player3_cards_coloured.append(cards_ordered_df.loc[2]["image_coloured"])
            player4_cards_coloured.append(cards_ordered_df.loc[3]["image_coloured"])

            player1_cards_binary.append(cards_ordered_df.loc[0]["image"])
            player2_cards_binary.append(cards_ordered_df.loc[1]["image"])
            player3_cards_binary.append(cards_ordered_df.loc[2]["image"])
            player4_cards_binary.append(cards_ordered_df.loc[3]["image"])


        player_cards_df = pd.DataFrame()

        player_cards_df["P1_coloured"] = pd.Series(player1_cards_coloured)
        player_cards_df["P2_coloured"] = pd.Series(player2_cards_coloured)
        player_cards_df["P3_coloured"] = pd.Series(player3_cards_coloured)
        player_cards_df["P4_coloured"] = pd.Series(player4_cards_coloured)

        player_cards_df["P1_binary"] = pd.Series(player1_cards_binary)
        player_cards_df["P2_binary"] = pd.Series(player2_cards_binary)
        player_cards_df["P3_binary"] = pd.Series(player3_cards_binary)
        player_cards_df["P4_binary"] = pd.Series(player4_cards_binary)

        training_list = []
        true_df = pd.read_csv(f"data/train_games/game{game_number+1}/game{game_number+1}.csv")
        for i in range (13):
            training_list.append((true_df.loc[i,"P1"],
                                  create_mnist_number_from_extracted_coloured_cards(player_cards_df.loc[i, "P1_coloured"],player_cards_df.loc[i, "P1_binary"]),
                                  create_mnist_number_from_extracted_binary_cards(player_cards_df.loc[i, "P1_binary"])))

            training_list.append((true_df.loc[i,"P2"],
                                  create_mnist_number_from_extracted_coloured_cards(player_cards_df.loc[i, "P2_coloured"],player_cards_df.loc[i, "P2_binary"]),
                                  create_mnist_number_from_extracted_binary_cards(player_cards_df.loc[i, "P2_binary"])))

            training_list.append((true_df.loc[i,"P3"],
                                  create_mnist_number_from_extracted_coloured_cards(player_cards_df.loc[i, "P3_coloured"],player_cards_df.loc[i, "P3_binary"]),
                                  create_mnist_number_from_extracted_binary_cards(player_cards_df.loc[i, "P3_binary"])))

            training_list.append((true_df.loc[i,"P4"],
                                  create_mnist_number_from_extracted_coloured_cards(player_cards_df.loc[i, "P4_coloured"],player_cards_df.loc[i, "P4_binary"]),
                                  create_mnist_number_from_extracted_binary_cards(player_cards_df.loc[i, "P4_binary"])))

        
        training_seriesj = pd.Series(training_list)
        final_series=pd.concat([final_series,training_seriesj])
        print(f"game{game_number+1}: registered")

    final_series.reset_index().to_pickle(f'data/classification_data/all_games_classification_series_mnist.pickle')
    
    

    return final_series
  

#Input: dict_data
#Output: Series with each element being a tuple ("3H", image_coloured, binary_image)
def create_semi_training_set_for_suits(dict_data):
    
    final_series = pd.Series()
    for game_number in range(7):
        player1_cards_coloured = []
        player2_cards_coloured = []
        player3_cards_coloured = []
        player4_cards_coloured = []

        player1_cards_binary = []
        player2_cards_binary = []
        player3_cards_binary = []
        player4_cards_binary = []

        for i in range(13):
            coloured_image = dict_data[f'game{game_number+1}'][f'round{i+1}']

            df_objs = find_potential_objects_in_original_round_image(coloured_image)
            cards_only_df = select_cards_from_potential_objects(df_objs)      
            cards_ordered_df = extract_cards_from_original_round_image(coloured_image, cards_only_df)


            player1_cards_coloured.append(cards_ordered_df.loc[0]["image_coloured"])
            player2_cards_coloured.append(cards_ordered_df.loc[1]["image_coloured"])
            player3_cards_coloured.append(cards_ordered_df.loc[2]["image_coloured"])
            player4_cards_coloured.append(cards_ordered_df.loc[3]["image_coloured"])

            player1_cards_binary.append(cards_ordered_df.loc[0]["image"])
            player2_cards_binary.append(cards_ordered_df.loc[1]["image"])
            player3_cards_binary.append(cards_ordered_df.loc[2]["image"])
            player4_cards_binary.append(cards_ordered_df.loc[3]["image"])


        player_cards_df = pd.DataFrame()

        player_cards_df["P1_coloured"] = pd.Series(player1_cards_coloured)
        player_cards_df["P2_coloured"] = pd.Series(player2_cards_coloured)
        player_cards_df["P3_coloured"] = pd.Series(player3_cards_coloured)
        player_cards_df["P4_coloured"] = pd.Series(player4_cards_coloured)

        player_cards_df["P1_binary"] = pd.Series(player1_cards_binary)
        player_cards_df["P2_binary"] = pd.Series(player2_cards_binary)
        player_cards_df["P3_binary"] = pd.Series(player3_cards_binary)
        player_cards_df["P4_binary"] = pd.Series(player4_cards_binary)

        training_list = []
        true_df = pd.read_csv(f"data/train_games/game{game_number+1}/game{game_number+1}.csv")
        for i in range (13):
            training_list.append((true_df.loc[i,"P1"],
                                  player_cards_df.loc[i, "P1_coloured"],
                                  player_cards_df.loc[i, "P1_binary"]))

            training_list.append((true_df.loc[i,"P2"],
                                  player_cards_df.loc[i, "P2_coloured"],
                                  player_cards_df.loc[i, "P2_binary"]))

            training_list.append((true_df.loc[i,"P3"],
                                  player_cards_df.loc[i, "P3_coloured"],
                                  player_cards_df.loc[i, "P3_binary"]))

            training_list.append((true_df.loc[i,"P4"],
                                  player_cards_df.loc[i, "P4_coloured"],
                                  player_cards_df.loc[i, "P4_binary"]))
        
        training_seriesj = pd.Series(training_list)
        final_series=pd.concat([final_series, training_seriesj])
        print(f"game{game_number+1}: registered")

    final_series.reset_index().to_pickle(f'data/classification_data/all_games_classification_series_semi_suits.pickle')
    
    return final_series
  

#Input: semi_suits_df
#Output: Series with each element being a tuple ("3H", suit_in_mnist_format)
def create_training_set_for_suits(semi_suits_df):
    semi_suits_df  = semi_suits_df.reset_index()
    training_list = []
    for i in range(364):
        coloured_card = semi_suits_df.iloc[i][0][1]
        suit_upper_left = create_mnist_suit_from_extracted_coloured_cards(coloured_card, True)
        suit_lower_right = create_mnist_suit_from_extracted_coloured_cards(coloured_card, False)
        suit_name = semi_suits_df.iloc[i][0][0]
        training_list.append((suit_name, suit_upper_left))
        training_list.append((suit_name, rotate(suit_lower_right, 180)))

    training_series = pd.Series(training_list)
    training_series.to_pickle(f'data/classification_data/all_games_classification_series_728_suits.pickle')
    return training_series

### Task 0

### HSV exploration for 2 random round images

In [ ]:
plot_round_image_in_HSV(dict_data['game2']['round1'])

In [ ]:
plot_round_image_in_HSV(dict_data['game6']['round4'])

To detect the cards and the dealer, we transformed the RGB image into a HSV image. By exploring the images, we discovered that the backgrounds of the cards and the D of the dealer coin had more or less the same HSV values (purple areas):
H: between 145 and 200 |
S: between 0 and 255 |
V: between ~170 and 255.

Our segmentation method called find_potential_objects_in_original_round_image() uses these ranges of values to detect the cards and the dealer (+ some noise objects that are removed later)

### Segmentation results for 2 random round images

In [ ]:
result1 = plot_overlay_for_round_image(dict_data['game1']['round1'])

In [ ]:
result2 = plot_overlay_for_round_image(dict_data['game1']['round2'])

In [ ]:
###################### PIPELINE FOR EXAM ##########################
path_to_image_folder = "TO BE FILLED"
dict_data_test = {}
dict_data_test["game1"] = {}
for i in range(13):
    dict_data['game1'][f'round{i+1}'] = skimage.io.imread(f"{path_to_image_folder}/{i+1}.jpg")


def plot_overlay_for_all_round_images(dict_data_test, game_number=1):
    
    for i in range(13):
        coloured_image = dict_data[f"game{game_number}"][f"round{i+1}"]
        print(f"The segmentation of cards and dealer for round {i+1} is:")
        plot_overlay_for_round_image(coloured_image)
    

### Pipeline: show segmentation for task 0

In [ ]:
results_df = plot_overlay_for_all_round_images(dict_data, game_number=1)

In [ ]:
results_df

In [ ]:
def transform_results_df_into_mnist_cards(results_df):
    results_df["P1_mnist_number"] = results_df["P1_extracted_card"].apply(lambda card: create_mnist_number_from_extracted_coloured_cards(card[0], card[1]))
    results_df["P2_mnist_number"] = results_df["P2_extracted_card"].apply(lambda card: create_mnist_number_from_extracted_coloured_cards(card[0], card[1]))
    results_df["P3_mnist_number"] = results_df["P3_extracted_card"].apply(lambda card: create_mnist_number_from_extracted_coloured_cards(card[0], card[1]))
    results_df["P4_mnist_number"] = results_df["P4_extracted_card"].apply(lambda card: create_mnist_number_from_extracted_coloured_cards(card[0], card[1]))

    results_df["P1_mnist_suit"] = results_df["P1_extracted_card"].apply(lambda card: create_mnist_suit_from_extracted_coloured_cards(card[0], upper_left=True))
    results_df["P2_mnist_suit"] = results_df["P2_extracted_card"].apply(lambda card: create_mnist_suit_from_extracted_coloured_cards(card[0], upper_left=True))
    results_df["P3_mnist_suit"] = results_df["P3_extracted_card"].apply(lambda card: create_mnist_suit_from_extracted_coloured_cards(card[0], upper_left=True))
    results_df["P4_mnist_suit"] = results_df["P4_extracted_card"].apply(lambda card: create_mnist_suit_from_extracted_coloured_cards(card[0], upper_left=True))

    return results_df

In [ ]:
mnist_df  = transform_results_df_into_mnist_cards(results_df)

In [ ]:
mnist_df.columns

In [ ]:
mnist_df

In [ ]:
# DEF Function PREDICT_NUMBER_FROM_MNIST
# DEF Function PREDICT_SUIT_FROM_MNIST

In [ ]:
def predict_number_from_mnist(mnist_image):
    return "4"

def predict_suit_from_mnist(mnist_image):
    return "H"

In [ ]:
def create_final_predicted_df(mnist_df):
    mnist_df = mnist_df[["P1_mnist_number","P2_mnist_number","P3_mnist_number","P4_mnist_number","P1_mnist_suit","P2_mnist_suit","P3_mnist_suit","P4_mnist_suit", "dealer"]]
    
    mnist_df["P1_mnist_number"] = mnist_df["P1_mnist_number"].apply(lambda mnist: predict_number_from_mnist(mnist))
    mnist_df["P2_mnist_number"] = mnist_df["P2_mnist_number"].apply(lambda mnist: predict_number_from_mnist(mnist))
    mnist_df["P3_mnist_number"] = mnist_df["P3_mnist_number"].apply(lambda mnist: predict_number_from_mnist(mnist))
    mnist_df["P4_mnist_number"] = mnist_df["P4_mnist_number"].apply(lambda mnist: predict_number_from_mnist(mnist))

    mnist_df["P1_mnist_suit"] = mnist_df["P1_mnist_suit"].apply(lambda mnist: predict_suit_from_mnist(mnist))
    mnist_df["P2_mnist_suit"] = mnist_df["P2_mnist_suit"].apply(lambda mnist: predict_suit_from_mnist(mnist))
    mnist_df["P3_mnist_suit"] = mnist_df["P3_mnist_suit"].apply(lambda mnist: predict_suit_from_mnist(mnist))
    mnist_df["P4_mnist_suit"] = mnist_df["P4_mnist_suit"].apply(lambda mnist: predict_suit_from_mnist(mnist))

    return mnist_df
    

In [ ]:
mnist_df_with_predictions = create_final_predicted_df(mnist_df)

In [ ]:
mnist_df_with_predictions

### Pipeline: Predict card number and suit for each player as well as dealer (task 1 & 2)

In [ ]:
# give coloured image of round
# retreive ordered_card_df + dealer_status